In [91]:
import pandas as pd
import lxml


## Reading our Notion Database Export

In [135]:
df = pd.read_html('data/Chicago Oy 2d952.html')[0]
df = df[df['Tags'] == 'Confirmed'].reset_index(drop=True)
df.head()

,Name,Time,Price per Oyster,Other Notable Specials,Location,Tags,Neighborhood
0,Swift and Sons,M-F 4-6:30pm,1.25,$5 beers / $8 wine / $10 daily cocktails,"1000 W Fulton Market, Chicago, IL 60607",Confirmed,West Loop
1,The Publican,M-F 4-6pm,2.00,Other appetizers and drink specials,"837 W Fulton Market Chicago, IL 60607",Confirmed,West Loop
2,Prime & Provisions,M-F 4-6pm,1.00,NaN,"222 N LaSalle St Chicago, IL 60601",Confirmed,Loop
3,Fisk & Co.,M-Thurs 4-6pm,2.00,$6 glasses of wine / Maximum 24 oysters per pe...,"225 N Wabash Ave Chicago, IL 60601",Confirmed,Loop
4,"Venteux Brasserie, Cafe & Oyster Bar",Tues-Fri. 4-6pm.,1.00,NaN,"224 N Michigan Ave Chicago, IL 60601",Confirmed,Loop


## Creating a Distance Matrix

We are particularly interested in distance from one location to another. I am planning on taking a Lyft to these locations nad exploring on foot since most of the happy hour specials also include alcohol and better to be safe than sorry. Because of this, we will need to find the walking distances from each location to each other one. For this we can use the Google Distance Matrix API to get our distance grid in walking minutes from each location to each other location.

In [109]:
import googlemaps
import dotenv
import os

In [110]:
dotenv.load_dotenv('.env')

True

In [111]:
gmaps = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))

After some tinkering it turns out that even though the **Distance Matrix API** limit for *origins* and *destinations* is 15 each, there is also a limit on the number of total calls, which is 100. Because of this your *origin x destination* matrix cannot exceed 100 elements...... A little annoying, but nothing some Python code cannot fix.

Below I've implemented a **ScalableDistanceMatrix** class that keeps track of our larger matrix and breaks down into the optimal number of API calls by minimizing our requests. If we treat our larger matrix as wanting to find walking distance between every location to one another, we just iterate through each row with the offset of our **MAX_LOCATIONS** value, which in this case is set to 10.

In [112]:
class ScalableDistanceMatrix:

    MAX_LOCATIONS = 10

    def __init__(self, client: googlemaps.Client, locations: list, destinations: list, mode='walking') -> None:
        self.client = client
        self.origins = locations
        self.destinations = locations
        self.mode = mode
        self.matrix = None
    
    def get_matrix(self) -> pd.DataFrame:
        if self.matrix is None:
            self._get_matrix()
        return self.matrix

    def _get_sub_matrices(self) -> list:
        out = []
        for row in range(0, (len(self.origins) - 1) // self.MAX_LOCATIONS + 1):
            for col in range(0, (len(self.destinations) - 1) // self.MAX_LOCATIONS + 1):
                # row and col are the indices of our larger matrix calls
                origins = self.origins[row*self.MAX_LOCATIONS:row*self.MAX_LOCATIONS + self.MAX_LOCATIONS]
                destinations = self.destinations[col*self.MAX_LOCATIONS:col*self.MAX_LOCATIONS + self.MAX_LOCATIONS]
                out.append((origins, destinations))
        return out

           
    def _get_matrix(self) -> None:
        self.matrix = pd.DataFrame(index = self.origins, columns = self.destinations)

        for sub_origins, sub_destinations in self._get_sub_matrices():
            matrix = self.client.distance_matrix(sub_origins, sub_destinations, mode=self.mode)
            for i, origin in enumerate(sub_origins):
                for j, destination in enumerate(sub_destinations):
                    self.matrix.loc[origin, destination] = matrix['rows'][i]['elements'][j]['duration']['value']
        
        return


In [113]:
client_walking = ScalableDistanceMatrix(gmaps, df['Location'].to_list(), df['Location'].to_list(), mode='walking')
client_driving = ScalableDistanceMatrix(gmaps, df['Location'].to_list(), df['Location'].to_list(), mode='driving')

In [114]:
df_res_walk = client_walking.get_matrix()
df_res_drive = client_driving.get_matrix()

Our response comes back in seconds, so for helping human reading, we can do a quick view of looking at it in terms of minutes.

In [115]:
df_res_walk / 60.0

,"1000 W Fulton Market, Chicago, IL 60607","837 W Fulton Market Chicago, IL 60607","222 N LaSalle St Chicago, IL 60601","225 N Wabash Ave Chicago, IL 60601","224 N Michigan Ave Chicago, IL 60601","350 N State St Chicago, IL 60654","400 N Clark St Chicago, IL 60654","In the alley behind Frontera Grill, 443 N Clark St Chicago, IL 60654","60 E Grand Ave Chicago, IL 60611","875 N Michigan Ave Chicago, IL 60610","2033 W North Ave Chicago, IL 60647","2125 W Division Chicago, IL 60622","2700 W Chicago Ave Chicago, IL 60622"
"1000 W Fulton Market, Chicago, IL 60607",0.0,4.366667,23.633333,31.25,32.8,30.416667,26.866667,28.433333,35.7,47.733333,50.016667,48.716667,53.533333
"837 W Fulton Market Chicago, IL 60607",4.333333,0.0,19.466667,27.083333,28.633333,26.383333,22.833333,24.4,31.75,43.7,49.416667,48.116667,57.8
"222 N LaSalle St Chicago, IL 60601",23.383333,19.233333,0.0,8.25,9.8,8.65,6.116667,7.666667,15.55,26.583333,64.2,62.9,72.783333
"225 N Wabash Ave Chicago, IL 60601",30.966667,26.833333,8.233333,0.0,2.55,6.35,9.933333,11.2,9.183333,19.75,71.783333,70.483333,80.383333
"224 N Michigan Ave Chicago, IL 60601",32.483333,28.333333,9.733333,2.5,0.0,7.6,11.45,12.7,9.433333,18.683333,73.3,72.0,81.883333
"350 N State St Chicago, IL 60654",30.133333,26.133333,8.633333,6.383333,7.616667,0.0,4.266667,5.5,6.85,18.683333,66.2,64.9,74.8
"400 N Clark St Chicago, IL 60654",26.733333,22.733333,6.183333,10.016667,11.566667,4.4,0.0,1.566667,9.266667,21.2,62.8,61.5,71.4
"In the alley behind Frontera Grill, 443 N Clark St Chicago, IL 60654",28.383333,24.383333,7.85,11.366667,12.916667,5.733333,1.65,0.0,7.7,19.75,62.933333,61.633333,71.55
"60 E Grand Ave Chicago, IL 60611",35.683333,31.75,15.716667,9.433333,9.5,7.066667,9.4,7.733333,0.0,12.4,67.283333,65.983333,75.9
"875 N Michigan Ave Chicago, IL 60610",47.7,43.7,26.766667,19.95,18.916667,19.516667,21.733333,20.083333,12.666667,0.0,67.816667,66.516667,78.1


In [116]:
df_res_drive / 60.0

,"1000 W Fulton Market, Chicago, IL 60607","837 W Fulton Market Chicago, IL 60607","222 N LaSalle St Chicago, IL 60601","225 N Wabash Ave Chicago, IL 60601","224 N Michigan Ave Chicago, IL 60601","350 N State St Chicago, IL 60654","400 N Clark St Chicago, IL 60654","In the alley behind Frontera Grill, 443 N Clark St Chicago, IL 60654","60 E Grand Ave Chicago, IL 60611","875 N Michigan Ave Chicago, IL 60610","2033 W North Ave Chicago, IL 60647","2125 W Division Chicago, IL 60622","2700 W Chicago Ave Chicago, IL 60622"
"1000 W Fulton Market, Chicago, IL 60607",0.0,3.5,7.766667,10.05,10.066667,9.833333,8.333333,8.7,10.466667,15.116667,9.9,11.05,12.716667
"837 W Fulton Market Chicago, IL 60607",2.75,0.0,6.083333,8.366667,9.183333,8.2,6.75,7.316667,9.616667,14.266667,10.233333,10.75,13.383333
"222 N LaSalle St Chicago, IL 60601",8.333333,7.05,0.0,3.55,4.466667,3.783333,3.966667,4.85,5.2,10.45,11.5,12.016667,14.866667
"225 N Wabash Ave Chicago, IL 60601",10.25,8.383333,3.183333,0.0,1.283333,3.333333,3.516667,4.583333,3.566667,7.65,13.116667,13.633333,16.483333
"224 N Michigan Ave Chicago, IL 60601",8.683333,7.383333,4.25,1.883333,0.0,1.516667,2.916667,4.75,1.45,7.183333,11.833333,12.366667,15.2
"350 N State St Chicago, IL 60654",9.8,7.933333,2.733333,1.8,2.616667,0.0,3.066667,4.133333,3.35,8.6,12.666667,13.183333,16.033333
"400 N Clark St Chicago, IL 60654",8.15,6.7,1.833333,3.15,3.966667,1.383333,0.0,1.75,2.8,8.466667,11.05,11.566667,14.416667
"In the alley behind Frontera Grill, 443 N Clark St Chicago, IL 60654",8.783333,7.283333,4.583333,5.483333,6.316667,3.683333,4.366667,0.0,4.333333,10.116667,10.783333,11.3,14.133333
"60 E Grand Ave Chicago, IL 60611",9.6,8.1,4.516667,2.733333,3.55,2.3,3.716667,5.183333,0.0,7.016667,11.55,12.083333,14.916667
"875 N Michigan Ave Chicago, IL 60610",13.75,12.683333,7.633333,5.716667,5.1,7.133333,7.983333,9.033333,5.45,0.0,14.7,15.216667,18.066667


## Location Groupings

In order to help us visualize what might be logical groupings of locations that are near one another, we can 

In [117]:
import plotly.figure_factory as ff

Because of the Google API calls, our matrix isn't necessarily exactly symmetric, but for the sake of creating our dendrogram, let's assume that walking time should be symmetric. Let's also fully convert our results into minutes.

In [118]:
def make_symmetric(df: pd.DataFrame, convert_seconds: bool = True) -> pd.DataFrame:
    df_out = df.copy()
    if convert_seconds:
        df_out = df_out / 60.0
    for i in range(len(df_out)):
        for j in range(i):
            df_out.iloc[i,j] = df_out.iloc[j,i]
    return df_out

df_res_drive = make_symmetric(df_res_drive)
df_res_walk = make_symmetric(df_res_walk)

In order to visualize our happy hours we may be able to easily group together, we can use a dendrogram. Dendrograms visualize hierarchical clustering groups of items that are most similar to one another. In our case, our distance matrix should be a fine feature vector for each location.

We draw out our dendrogram and also set a color threshold of 30 signifying that if there are two locations within 30 minutes walking distance to one another, they will be drawn with the same color in adjacent groupings. 30 minutes should be enough time to enjoy oysters, take down my notes, snap a couple of photos if we aim for a max of 3-4 locations a day where we only want to call a Lyft one time there and back and the average happy hour is 2.5 hours.

In [203]:
fig = ff.create_dendrogram(
        df_res_walk.values.astype(float), 
        orientation='left', 
        labels=df['Name'].to_list(),
        color_threshold=30,
        )
fig.update_layout(width=1000, height=800, title_text='Oyster Happy Hour Groups (Walking)', title_x=0.5)
fig.show()

Let's also make a dendrogram with 10 minutes being the color grouping for the driving times.

In [204]:
fig = ff.create_dendrogram(
        df_res_drive.values.astype(float), 
        orientation='left', 
        labels=df['Name'].to_list(),
        color_threshold=10,
        )
fig.update_layout(width=1000, height=800, title_text='Oyster Happy Hour Groups (Driving)', title_x=0.5)
fig.show()

This is a great start to mapping out the ideal trajectory. However, we now have to deal with the fact that these happy hours can be on different days and have slightly different time windows. This may be problematic and we may have to adjust our initial naive distance based groups above. Let's try and do some visualizations based on thes available timing intervals. We want to bring in cluster groupings from above to color on top of our time intervals. Since plotly is using hierarchical clustering to draw its plots, we'll just use sklearn's AgglomerativeClustering algorithm to get similar clusterings.

In [186]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=6, linkage='single')
df['walking_cluster'] = cluster.fit_predict(df_res_walk.values.astype(float))
df['driving_cluster'] = cluster.fit_predict(df_res_drive.values.astype(float))

In [188]:
df[['Name', 'walking_cluster', 'driving_cluster']]

,Name,walking_cluster,driving_cluster
0,Swift and Sons,2,1
1,The Publican,2,1
2,Prime & Provisions,0,5
3,Fisk & Co.,0,3
4,"Venteux Brasserie, Cafe & Oyster Bar",0,3
5,Tortoise Supper Club,0,3
6,The Smith,0,3
7,Bar Sotano,0,3
8,Joe’s Seafood,0,3
9,The Signature Lounge,4,2


In order to visualize the timing intervals, we'll bring in a table that has a little bit neater data that was organized from the master table. This table has information on each day and the specific hours for those days for each location.

In [149]:
df_inters = pd.read_html('data/Chicago Oy 2d952.html')[1]

In [150]:
df_inters.head()

,Name,Interval,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Swift and Sons,16:00-18:30,Yes,Yes,Yes,Yes,Yes,NaN,NaN
1,The Publican,16:00-18:00,Yes,Yes,Yes,Yes,Yes,NaN,NaN
2,Prime & Provisions,16:00-18:00,Yes,Yes,Yes,Yes,Yes,NaN,NaN
3,Fisk & Co.,16:00-18:00,Yes,Yes,Yes,Yes,NaN,NaN,NaN
4,"Venteux Brasserie, Cafe & Oyster Bar",16:00-18:00,NaN,Yes,Yes,Yes,Yes,NaN,NaN


In [151]:
from datetime import datetime
import hashlib

In [198]:

df_inters['Start'] = df_inters['Interval'].apply(lambda x: datetime(2020, 1, 1, int(x.split('-')[0].split(':')[0]), int(x.split('-')[0].split(':')[1])))
df_inters['Finish'] = df_inters['Interval'].apply(lambda x: datetime(2020, 1, 1, int(x.split('-')[1].split(':')[0]), int(x.split('-')[1].split(':')[1])))
df_inters['Task'] = df_inters['Name']




# We need to have a way to color by our clusters from above
def colFromStr(inputString):
    hashedString = hashlib.sha256(inputString.encode())
    return hashedString.hexdigest()[len(hashedString.hexdigest())-6:] 

colours = {}
for name in df_inters['Name'].unique():
    key = f"Cluster {df[df['Name'] == name]['driving_cluster'].values[0]}"
    colour = colFromStr(str(key))
    colours[name] = f"#{colour}"

def draw_gantt_day(df: pd.DataFrame, day: str) -> None:
    if day not in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        raise Exception('Invalid day')
    df_day = df[df[day] == 'Yes']
    fig = ff.create_gantt(df_day, index_col='Task', colors=colours,
                            #show_colorbar=True, 
                            #group_tasks=True, 
                            showgrid_x=True, showgrid_y=True, 
                            title=f'Oyster Happy Hour {day}')
    fig.show()
    return



In [205]:
for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    draw_gantt_day(df_inters, day)
